In [ ]:
# Import all the tools we need
import numpy as np
import pandas as pd
from sklearn.covariance import EllipticEnvelope
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM, LinearSVC, SVC, SVR
from sklearn.metrics import accuracy_score

In [ ]:
pursuit = pd.concat(map(pd.read_csv, ["D:\\ELSEC\\SortiModel\\data\\1.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\2.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\3.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\4.csv"]), ignore_index=True)
clutter = pd.concat(map(pd.read_csv, ["D:\\ELSEC\\SortiModel\\data\\1.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\2.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\3.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\4.csv"]), ignore_index=True)
print(f'Size of pursuit signals: {pursuit.shape}')
print(f'Size of clutter signals: {clutter.shape}')
pursuit = pursuit.drop(pursuit[pursuit.WeaponSystem == 'T5'].index)
clutter = clutter.drop(clutter[clutter.WeaponSystem != 'T5'].index)

pursuit.loc[pursuit.WeaponSystem == 'T5', 'Target'] = 0
pursuit.loc[pursuit.WeaponSystem != 'T5', 'Target'] = 1
pursuit.loc[pursuit['Target'] > 1.0, 'Target'] = 1
pursuit['Target'] = pursuit['Target'].fillna(0)

clutter.loc[clutter.WeaponSystem == 'T5', 'Target'] = 0
clutter.loc[clutter.WeaponSystem != 'T5', 'Target'] = 1
clutter.loc[clutter['Target'] > 1.0, 'Target'] = 1
clutter['Target'] = clutter['Target'].fillna(0)
pursuit['DF Type'] = pursuit['DF Type'].replace('BLI1', 1)
pursuit['DF Type'] = pursuit['DF Type'].replace('ADF', 2)
pursuit['DF Type'] = pursuit['DF Type'].replace('BLI2', 3)
pursuit['DF Type'] = pursuit['DF Type'].replace('ADF+BLI2', 4)
pursuit['DF Type'] = pursuit['DF Type'].replace('ADF+BLI1', 5)
pursuit['DF Type'].astype(int)

pursuit['Frequency Agility'] = pursuit['Frequency Agility'].replace('Fixed Frequency', 1)
pursuit['Frequency Agility'] = pursuit['Frequency Agility'].replace('LFM', 2)
pursuit['Frequency Agility'] = pursuit['Frequency Agility'].replace('LFM (Chirp-Up)', 3)
pursuit['Frequency Agility'] = pursuit['Frequency Agility'].replace('Frequency Agile', 4)
pursuit['Frequency Agility'] = pursuit['Frequency Agility'].replace("LFM (Chirp-Up) / Frequency Agile", 5)
pursuit['Frequency Agility'] = pursuit['Frequency Agility'].replace('LFM / Frequency Agile', 6)
pursuit['Frequency Agility'].astype(int)

pursuit['Pulse Agility'] = pursuit['Pulse Agility'].replace('Fixed PRI', 1)
pursuit['Pulse Agility'] = pursuit['Pulse Agility'].replace('Fixed Frequency', 2)
pursuit['Pulse Agility'] = pursuit['Pulse Agility'].replace('Stagger PRI', 3)
pursuit['Pulse Agility'] = pursuit['Pulse Agility'].replace('Dwell&Switch PRI', 4)
pursuit['Pulse Agility'] = pursuit['Pulse Agility'].replace('Frequency Agile', 5)
pursuit['Pulse Agility'] = pursuit['Pulse Agility'].replace('LFM', 5)
pursuit['Pulse Agility'].astype(int)

pursuit['TOLA'] = pursuit['TOLA'].str.replace(' ', '').str.split(':').apply(lambda x: int(x[0])*3600 + int(x[1])*60 + int(x[2]))
pursuit['TOLA'] = pursuit['TOLA'].astype(int)
pursuit['TOFA'] = pursuit['TOFA'].str.replace(' ', '').str.split(':').apply(lambda x: int(x[0])*3600 + int(x[1])*60 + int(x[2]))
pursuit['TOFA'] = pursuit['TOFA'].astype(int)

pursuit['TD'] = (pursuit['TOLA'] - pursuit['TOFA'])
pursuit['TD'].info()

pursuit_t1 =  pursuit.drop(pursuit[pursuit.WeaponSystem != 'T1'].index)
pursuit_t2 =  pursuit.drop(pursuit[pursuit.WeaponSystem != 'T2'].index)
pursuit_t3 =  pursuit.drop(pursuit[pursuit.WeaponSystem != 'T3'].index)
pursuit_t4 =  pursuit.drop(pursuit[pursuit.WeaponSystem != 'T4'].index)

pursuit = pursuit.select_dtypes(exclude='object')
pursuit = pursuit.drop(['Emitter Name'], axis=1)
print('Pursuit Dataset Revision done !')

pursuit_t1 = pursuit_t1.select_dtypes(exclude='object')
pursuit_t1 = pursuit_t1.drop(['Emitter Name'], axis=1)
pursuit_t2 = pursuit_t2.select_dtypes(exclude='object')
pursuit_t2 = pursuit_t2.drop(['Emitter Name'], axis=1)
pursuit_t3 = pursuit_t3.select_dtypes(exclude='object')
pursuit_t3 = pursuit_t3.drop(['Emitter Name'], axis=1)
pursuit_t4 = pursuit_t4.select_dtypes(exclude='object')
pursuit_t4 = pursuit_t4.drop(['Emitter Name'], axis=1)

clutter['DF Type'] = clutter['DF Type'].replace('BLI1', 1)
clutter['DF Type'] = clutter['DF Type'].replace('ADF', 2)
clutter['DF Type'] = clutter['DF Type'].replace('BLI2', 3)
clutter['DF Type'] = clutter['DF Type'].replace('ADF+BLI2', 4)
clutter['DF Type'] = clutter['DF Type'].replace('ADF+BLI1', 5)
clutter['DF Type'].astype(int)

clutter['Frequency Agility'] = clutter['Frequency Agility'].replace('Fixed Frequency', 1)
clutter['Frequency Agility'] = clutter['Frequency Agility'].replace('LFM', 2)
clutter['Frequency Agility'] = clutter['Frequency Agility'].replace('LFM (Chirp-Up)', 3)
clutter['Frequency Agility'] = clutter['Frequency Agility'].replace('Frequency Agile', 4)
clutter['Frequency Agility'] = clutter['Frequency Agility'].replace("LFM (Chirp-Up) / Frequency Agile", 5)
clutter['Frequency Agility'] = clutter['Frequency Agility'].replace('LFM / Frequency Agile', 6)
clutter['Frequency Agility'].astype(int)

clutter['Pulse Agility'] = clutter['Pulse Agility'].replace('Fixed PRI', 1)
clutter['Pulse Agility'] = clutter['Pulse Agility'].replace('Fixed Frequency', 2)
clutter['Pulse Agility'] = clutter['Pulse Agility'].replace('Stagger PRI', 3)
clutter['Pulse Agility'] = clutter['Pulse Agility'].replace('Dwell&Switch PRI', 4)
clutter['Pulse Agility'] = clutter['Pulse Agility'].replace('Frequency Agile', 5)
clutter['Pulse Agility'] = clutter['Pulse Agility'].replace('LFM', 5)
clutter['Pulse Agility'].astype(int)


clutter['TOLA'] = clutter['TOLA'].str.replace(' ', '').str.split(':').apply(lambda x: int(x[0])*3600 + int(x[1])*60 + int(x[2]))
clutter['TOLA'] = clutter['TOLA'].astype(int)
clutter['TOFA'] = clutter['TOFA'].str.replace(' ', '').str.split(':').apply(lambda x: int(x[0])*3600 + int(x[1])*60 + int(x[2]))
clutter['TOFA'] = clutter['TOFA'].astype(int)
clutter['TD'] = (clutter['TOLA'] - clutter['TOFA'])
clutter['TD'].info()

clutter = clutter.select_dtypes(exclude='object')
clutter = clutter.drop(['Emitter Name'], axis=1)
print('Clutter Dataset Revision done !')
print('Dataset processing done !') 

In [ ]:
pursuit_test = pursuit.drop('Target', axis=1)
pursuit_res = pursuit['Target']
pursuit_t1_test = pursuit_t1.drop('Target', axis=1)
pursuit_t1_res = pursuit_t1['Target']
pursuit_t2_test = pursuit_t2.drop('Target', axis=1)
pursuit_t2_res = pursuit_t2['Target']
pursuit_t3_test = pursuit_t3.drop('Target', axis=1)
pursuit_t3_res = pursuit_t3['Target']
pursuit_t4_test = pursuit_t4.drop('Target', axis=1)
pursuit_t4_res = pursuit_t4['Target']

In [ ]:
print(f'Size of Pursuit T1:{len(pursuit_t1)}')
print(f'Size of Pursuit T2:{len(pursuit_t2)}')
print(f'Size of Pursuit T3:{len(pursuit_t3)}')
print(f'Size of Pursuit T4:{len(pursuit_t4)}')

In [ ]:
train_size = int(len(pursuit_t3) * 0.8)
X_train = pursuit_t3_test[:train_size]
y_train = pursuit_t3_res[:train_size]

X_test = pursuit_t3_test[train_size:]
y_test = pursuit_t3_res[train_size:]

y_train_np = np.array(y_train)
num_ones_train = (y_train_np == 1).sum()
print(f'Total Number of Ones in Target Column of the training dataset: {num_ones_train}')
print(f'Total Number of Zeros in Target Column of the training dataset: {len(y_train) - num_ones_train}')
y_test_np = np.array(y_test)
num_ones_test = (y_test_np == 1).sum()
print(f'Total Number of Ones in Target Column of the test dataset: {num_ones_test}')
print(f'Total Number of Zeros in Target Column of the test dataset: {len(y_test) - num_ones_test}')
num_ones_test + num_ones_train, (len(y_test) - num_ones_test) + (len(y_train) - num_ones_train)

In [ ]:
# One-Class SVM
model_svm_01 = OneClassSVM(kernel='poly', gamma=0.5, nu=0.9)
#model_svm_01 = EllipticEnvelope(contamination=0.1, random_state=42)
#model_svm_01 =  LocalOutlierFactor(n_neighbors=20, contamination="auto", novelty=True)
#model_svm_01 = SVC(kernel='poly', degree=3, gamma='scale')
#model_svm_01 = SVR(kernel='poly', gamma='auto', C=1.0, epsilon=12)
#model_svm_01 = SGDClassifier(loss='log_loss', penalty='l2')
# Fit the model on the training data

model_svm_01.fit(X_train, y_train)

# Predict on the test data
y_pred_svm = model_svm_01.predict(X_test)

# Convert the predictions to 1 (anomaly) and 0 (normal)
y_pred_svm = np.where(y_pred_svm == -1, 1, 0)

# Calculate and print the accuracy
accuracy_svm = accuracy_score(y_test, y_pred_svm)
#print(f'Prediction:', y_pred_svm)
print(f"Accuracy of One-Class SVM: {accuracy_svm}")

In [ ]:
# Predict on the test data
y_pred_svm = model_svm_01.predict(pursuit_t1_test)

# Convert the predictions to 1 (anomaly) and 0 (normal)
y_pred_svm = np.where(y_pred_svm == -1, 1, 0)

# Calculate and print the accuracy
accuracy_svm = accuracy_score(pursuit_t1_res, y_pred_svm)
#print(f'Prediction:', y_pred_svm)
print(f"Accuracy of One-Class SVM: {accuracy_svm}")

In [ ]:
# Predict on the test data
y_pred_svm = model_svm_01.predict(pursuit_t2_test)

# Convert the predictions to 1 (anomaly) and 0 (normal)
y_pred_svm = np.where(y_pred_svm == -1, 1, 0)

# Calculate and print the accuracy
accuracy_svm = accuracy_score(pursuit_t2_res, y_pred_svm)
#print(f'Prediction:', y_pred_svm)
print(f"Accuracy of One-Class SVM: {accuracy_svm}")

In [ ]:
# Predict on the test data
y_pred_svm = model_svm_01.predict(pursuit_t3_test)

# Convert the predictions to 1 (anomaly) and 0 (normal)
y_pred_svm = np.where(y_pred_svm == -1, 1, 0)

# Calculate and print the accuracy
accuracy_svm = accuracy_score(pursuit_t3_res, y_pred_svm)
#print(f'Prediction:', y_pred_svm)
print(f"Accuracy of One-Class SVM: {accuracy_svm}")

In [ ]:
# Predict on the test data
y_pred_svm = model_svm_01.predict(pursuit_t4_test)

# Convert the predictions to 1 (anomaly) and 0 (normal)
y_pred_svm = np.where(y_pred_svm == -1, 1, 0)

# Calculate and print the accuracy
accuracy_svm = accuracy_score(pursuit_t4_res, y_pred_svm)
#print(f'Prediction:', y_pred_svm)
print(f"Accuracy of One-Class SVM: {accuracy_svm}")

In [ ]:
clutter_X = clutter.drop("Target", axis=1)
clutter_y = clutter["Target"]

# Predict on the test data
y_pred_svm = model_svm_01.predict(clutter_X)

# Convert the predictions to 1 (anomaly) and 0 (normal)
y_pred_svm = np.where(y_pred_svm == -1, 1, 0)

# Calculate and print the accuracy
accuracy_svm = accuracy_score(clutter_y, y_pred_svm)
#print(f'Prediction:', y_pred_svm)
clutter['Target'] = y_pred_svm
clutter_X_arr = np.array(clutter_X)
print(f"Accuracy of One-Class SVM: {accuracy_svm}")
j = 0
for i in range(len(clutter_X_arr)):
    if y_pred_svm[i] == 1:
        j = j + 1
print("Total clutter", j)

In [ ]:
clutter['Target'].value_counts()

In [ ]:
df = pd.read_csv('ReportOnClutter.csv')

In [ ]:
df['Target'].value_counts()

In [ ]:
df[df['Duration Of Intercept'] == 0].describe()

In [ ]:
df[df['Target'] == 0].describe()